All Libraries to Install:

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [ ]:
with open("kdd_names.txt",'r') as f:
    print(f.read())

In [ ]:
with open("training_attack_types.txt",'r') as f:
    print(f.read())

In [ ]:
datacols =["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","target"]

In [ ]:
len(datacols)

In [ ]:
df = pd.read_csv("data10per.csv", header=None, names = datacols)
df.head()

In [ ]:
df['target'].value_counts()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df.isnull().sum()

In [ ]:
attacks_types = {'normal': 'normal','back': 'dos','buffer_overflow':'u2r','ftp_write': 'r2l','guess_passwd': 'r2l','imap': 'r2l','ipsweep':'probe','land': 'dos','loadmodule': 'u2r','multihop': 'r2l','neptune':'dos','nmap': 'probe','perl': 'u2r','phf': 'r2l','pod': 'dos','portsweep':'probe','rootkit': 'u2r','satan': 'probe','smurf': 'dos','spy':'r2l','teardrop': 'dos','warezclient': 'r2l','warezmaster': 'r2l',}

In [ ]:
df['Attack Type'] = df.target.apply(lambda r:attacks_types[r[:-1]])
df.head()

In [ ]:
df['Attack Type'].value_counts()

In [ ]:
df.isnull().sum()

In [ ]:
num_cols = df._get_numeric_data().columns
cate_cols = list(set(df.columns)-set(num_cols))
cate_cols.remove('target')
cate_cols.remove('Attack Type')
cate_cols

In [ ]:
def bar_graph(feature):
    df[feature].value_counts().plot(kind="bar")
    bar_graph('protocol_type')
    plt.figure(figsize=(15,3))  
    bar_graph('service')
    bar_graph('flag')
    df.columns
    df = df.dropna('columns')

 drop columns with NaN

In [ ]:
df = df[col for col in df if df[col].nunique() > 1]

keep columns where there are more

In [ ]:
corr = df.corr()
plt.figure(figsize=(15,12))
sns.heatmap(corr)
plt.show()

In [ ]:
df['num_root'].corr(df['num_compromised'])
df['srv_serror_rate'].corr(df['serror_rate'])
df['srv_count'].corr(df['count'])
df['srv_rerror_rate'].corr(df['rerror_rate'])
df['dst_host_srv_rerror_rate'].corr(df['rerror_rate'])
df['dst_host_srv_rerror_rate'].corr(df['srv_rerror_rate'])

These variables are highly correlated with another variable and should be
ignored for analysis.
(Correlation = 0.9938277978738366)

In [ ]:
df.drop('num_root',axis = 1,inplace = True)
df.drop('srv_serror_rate',axis = 1,inplace = True)
df.drop('srv_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_srv_serror_rate',axis = 1, inplace=True)
df.drop('dst_host_serror_rate',axis = 1, inplace=True)

In [ ]:
df.drop('dst_host_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_srv_rerror_rate',axis = 1, inplace=True)
df.drop('dst_host_same_srv_rate',axis = 1, inplace=True)
df.head()

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df_std = df.std()
df_std = df_std.sort_values(ascending = True)
df_std

In [ ]:
df['protocol_type'].value_counts()
pmap = {'icmp':0,'tcp':1,'udp':2}
df['protocol_type'] = df['protocol_type'].map(pmap)
df['flag'].value_counts()
fmap = {'SF':0,'S0':1,'REJ':2,'RSTR':3,'RSTO':4,'SH':5 ,'S1':6,'S2':7,'RSTOS0':8,'S3':9,'OTH':10}

In [ ]:
df['flag'] = df['flag'].map(fmap)
df.head()

In [ ]:
df.drop('service',axis = 1,inplace= True)
df.shape

In [ ]:
df.head()

In [ ]:
df.dtypes

MODELS

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
df = df.drop(['target',], axis=1)
print(df.shape)

Target variable and train set

In [ ]:
Y = df[['Attack Type']]
X = df.drop(['Attack Type',], axis=1)

In [ ]:
sc = MinMaxScaler()
X = sc.fit_transform(X)

Split test and train data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33,
random_state=42)
print(X_train.shape, X_test.shape)
print(Y_train.shape, Y_test.shape)

GAUSSIAN NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import GaussianNB
model1 = GaussianNB( )
start_time = time.time()

In [ ]:
model1.fit(X_train, Y_train.values.ravel())
end_time = time.time()
train_time_nb=end_time-start_time
print("Training time: ",train_time_nb)
start_time = time.time()

In [ ]:
Y_test_pred1 = model1.predict(X_test)
end_time = time.time()
test_time_nb=end_time-start_time
print("Testing time: ",test_time_nb)
train_score_nb=model1.score(X_train, Y_train)
test_score_nb=model1.score(X_test,Y_test)
print("Train score is:",train_score_nb)
print("Test score is:",test_score_nb)

DECISION TREE

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model2 = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
start_time = time.time()
model2.fit(X_train, Y_train.values.ravel())
end_time = time.time()
train_time_dt=end_time-start_time
print("Training time: ",train_time_dt)
start_time = time.time()
Y_test_pred2 = model2.predict(X_test)
end_time = time.time()
test_time_dt=end_time-start_time
print("Testing time: ",test_time_dt)
train_score_dt=model2.score(X_train, Y_train)
test_score_dt=model2.score(X_test,Y_test)
print("Train score is:", model2.score(X_train, Y_train))
print("Test score is:",model2.score(X_test,Y_test))

RANDOM FOREST

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
model2 = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
start_time = time.time()
model2.fit(X_train, Y_train.values.ravel())
end_time = time.time()
train_time_dt=end_time-start_time
print("Training time: ",train_time_dt)
start_time = time.time()
Y_test_pred2 = model2.predict(X_test)
end_time = time.time()
test_time_dt=end_time-start_time
print("Testing time: ",test_time_dt)
train_score_dt=model2.score(X_train, Y_train)
test_score_dt=model2.score(X_test,Y_test)
print("Train score is:", model2.score(X_train, Y_train))
print("Test score is:",model2.score(X_test,Y_test))

RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model3 = RandomForestClassifier(n_estimators=30)
start_time = time.time()
model3.fit(X_train, Y_train.values.ravel())
end_time = time.time()
train_time_rf=end_time-start_time
print("Training time: ",train_time_rf)
start_time = time.time()
Y_test_pred3 = model3.predict(X_test)
end_time = time.time()


In [ ]:
test_time_rf=end_time-start_time
print("Testing time: ",test_time_rf)
train_score_rf=model3.score(X_train, Y_train)
test_score_rf=model3.score(X_test,Y_test)
print("Train score is:",train_score_rf)
print("Test score is:",test_score_rf)

SUPPORT VECTOR MACHINE

In [ ]:
from sklearn.svm import SVC

In [ ]:
model4 = SVC(gamma = 'scale')
start_time = time.time()
model4.fit(X_train, Y_train.values.ravel())
end_time = time.time()
train_time_svm=end_time-start_time
print("Training time: ",train_time_svm)
start_time = time.time()
Y_test_pred4 = model4.predict(X_test)
end_time = time.time()
test_time_svm=end_time-start_time
print("Testing time: ",test_time_svm)
train_score_svm=model4.score(X_train, Y_train)
test_score_svm=model4.score(X_test,Y_test)
print("Train score is:", train_score_svm)
print("Test score is:", test_score_svm)

LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model5 = LogisticRegression(max_iter=1200000)
start_time = time.time()
model5.fit(X_train, Y_train.values.ravel())
end_time = time.time()
train_time_lr=end_time-start_time
print("Training time: ",train_time_lr)
start_time = time.time()
Y_test_pred5 = model5.predict(X_test)
end_time = time.time()


In [ ]:
test_time_lr=end_time-start_time
print("Testing time: ",test_time_lr)
train_score_lr=model5.score(X_train, Y_train)
test_score_lr=model5.score(X_test,Y_test)
print("Train score is:",train_score_lr)
print("Test score is:",test_score_lr)

GRADIENT BOOSTING CLASSIFIER

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
model6 = GradientBoostingClassifier(random_state=0)
start_time = time.time()
model6.fit(X_train, Y_train.values.ravel())
end_time = time.time()
train_time_gb=end_time-start_time
print("Training time: ",train_time_gb)
start_time = time.time()
Y_test_pred6 = model6.predict(X_test)
end_time = time.time()
test_time_gb=end_time-start_time
print("Testing time: ",test_time_gb)
train_score_gb=model6.score(X_train, Y_train)
test_score_gb=model6.score(X_test,Y_test)
print("Train score is:",train_score_gb)
print("Test score is:",test_score_gb)

ACCURACY(TRAINING)

In [ ]:
names = ['NB','DT','RF','SVM','LR','GB']
value =[train_score_nb,train_score_dt,train_score_rf,train_score_svm,train_score_lr,train_score_gb]
values=[]

In [ ]:
for i in value:
    a=i*100
    values.append(a)

In [ ]:
f = plt.figure(figsize=(15,3),num=10)
plt.subplot(131)
plt.ylim(85,102)
plt.bar(names,values)

In [ ]:
f.savefig('training_accuracy_figure.png',bbox_inches='tight')

ACCURACY(Testing)

In [ ]:
names = ['NB','DT','RF','SVM','LR','GB']
value =[test_score_nb,test_score_dt,test_score_rf,test_score_svm,test_score_lr,test_score_gb]
values=[]

In [ ]:
for i in value:
    a=i*100
    values.append(a)

In [ ]:
f = plt.figure(figsize=(15,3),num=10)
plt.subplot(131)
plt.ylim(85,102)
plt.bar(names,values)
f.savefig('test_accuracy_figure.png',bbox_inches='tight')

TIME(TRAINING)

In [ ]:
names = ['NB','DT','RF','SVM','LR','GB']
values =[train_time_nb,train_time_dt,train_time_rf,train_time_svm,train_time_lr,train_time_gb]
f = plt.figure(figsize=(15,3),num=10)
plt.subplot(131)
plt.bar(names,values)
f.savefig('train_time_figure.png',bbox_inches='tight')

TIME(TESTING)

In [ ]:
names = ['NB','DT','RF','SVM','LR','GB']
values =[test_time_nb,test_time_dt,test_time_rf,test_time_svm,test_time_lr,test_time_gb]
f = plt.figure(figsize=(15,3),num=10)
plt.subplot(131)
plt.bar(names,values)
f.savefig('test_time_figure.png',bbox_inches='tight')
